## Tweets extraction 

In [ ]:
!pip install tweepy datetime

## Play

In [ ]:
#!python get_tweets.py

## JSON structure

In [ ]:
import json

In [ ]:
# open file
with open("collected_tweets_2020-11-20-16-40-37.txt",'r') as file:
    tweets = file.readlines()

In [ ]:
with open('tweet.json', 'w') as f:
    json.dump(
        json.loads(json.loads(tweets[0])), f
    )

In [ ]:
json.loads(json.loads(tweets[0]))


In [ ]:
parsed_tweets = [json.loads(json.loads(i)) for i in tweets]

In [ ]:
#number of coleted tweets

len(parsed_tweets)

## Acessing JSON keys

In [ ]:
firsttweet = parsed_tweets[0]

In [ ]:
## tweet message
## first level key
firsttweet['text']

In [ ]:
## user key
firsttweet['user']['screen_name']

In [ ]:
## info of original tweet
firsttweet['retweeted_status']['created_at']

In [ ]:
## first name user of retweet
firsttweet['retweeted_status']['entities']['user_mentions']#[0]['name']

### Transform tweets in relational structure

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(firsttweet).reset_index(drop=True).iloc[:1]
df

In [ ]:
df.columns

In [ ]:
df.drop(columns = ['quote_count', 'reply_count', 'retweet_count',
       'favorite_count', 'favorited', 'retweeted', 'user', 'entities',
                  'retweeted_status',
                  ], inplace = True)

In [ ]:
df['user_id'] = firsttweet['user']['id']
df['user_id_str'] = firsttweet['user']['id_str']
df['user_screen_name'] = firsttweet['user']['screen_name']
df['user_location'] = firsttweet['user']['location']
df['user_description'] = firsttweet['user']['description']
df['user_protected'] = firsttweet['user']['protected']
df['user_verified'] = firsttweet['user']['verified']
df['user_followers_count'] = firsttweet['user']['followers_count']
df['user_friends_count'] = firsttweet['user']['friends_count']
df['user_created_at'] = firsttweet['user']['created_at']

In [ ]:
df

In [ ]:
user_mentions = []

for i in range(len(firsttweet['entities']['user_mentions'])):
    dict1 = firsttweet['entities']['user_mentions'][i].copy()
    dict1.pop('indices', None)
    df2 = pd.DataFrame(dict1, index= [0])
    df2 = df2.rename(columns = {
        'screen_name': 'entities_screen_name',
        'name': 'entities_name',
        'id': 'entities_id',
        'id_str': 'entities_id_str'
    })
    user_mentions.append(df2)

In [ ]:
user_mentions[0]

In [ ]:
pd.concat(user_mentions, ignore_index = True)

In [ ]:
dfs = []
for i in user_mentions:
    dfs.append(
        pd.concat([df.copy(), i], axis=1)
    )

In [ ]:
dfs[0]

In [ ]:
pd.concat(dfs, ignore_index=True)

### Creating funtion to all tweets based in development done before

In [ ]:
def tweet_to_df(tweet):
    try:
        df = pd.DataFrame(tweet).reset_index(drop=True).iloc[:1]
        df.drop(columns = ['quote_count', 'reply_count', 'retweet_count',
           'favorite_count', 'favorited', 'retweeted', 'user', 'entities',
                      'retweeted_status',
                      ], inplace = True)
        df['user_id'] = tweet['user']['id']
        df['user_id_str'] = tweet['user']['id_str']
        df['user_screen_name'] = tweet['user']['screen_name']
        df['user_location'] = tweet['user']['location']
        df['user_description'] = tweet['user']['description']
        df['user_protected'] = tweet['user']['protected']
        df['user_verified'] = tweet['user']['verified']
        df['user_followers_count'] = tweet['user']['followers_count']
        df['user_friends_count'] = tweet['user']['friends_count']
        df['user_created_at'] = tweet['user']['created_at']

        user_mentions = []

        for i in range(len(tweet['entities']['user_mentions'])):
            dict1 = tweet['entities']['user_mentions'][i].copy()
            dict1.pop('indices', None)
            df2 = pd.DataFrame(dict1, index= [0])
            df2 = df2.rename(columns = {
                'screen_name': 'entities_screen_name',
                'name': 'entities_name',
                'id': 'entities_id',
                'id_str': 'entities_id_str'
            })
            user_mentions.append(df2)

        dfs = []
        for i in user_mentions:
            dfs.append(
                pd.concat([df.copy(), i], axis=1)
            )
        df_final = pd.concat(dfs, ignore_index=True)
    except:
        return None
    
    return df_final

In [ ]:
tweet_to_df(parsed_tweets[100])

In [ ]:
%%time
# iteration to all tweets

fullparsed = [tweet_to_df(tweet) for tweet in parsed_tweets]

In [ ]:
# remove empty positions from list

fullparsed = [i for i in fullparsed if i is not None]

len(fullparsed)

In [ ]:
treated_df = pd.concat(fullparsed, ignore_index = True, sort=True)

In [ ]:
treated_df

### sql server ingestion

In [ ]:
import pyodbc
import sqlalchemy

In [ ]:
engine = sqlalchemy.create_engine('mssql+pyodbc://@localhost/twitter?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server')

In [ ]:
treated_df.to_sql("tweets", con=engine, index=False,if_exists="append")